# 杭州市萧山区项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`CHEM-Data`*

---
*@author: Evan*\
*@date: 2023-06-20*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
chem = xr.open_dataset('F:/Data/Project_Xiaoshan/compare_emis/heic_2022_chem.nc')
grid = xr.open_dataset('F:/Data/Project_Xiaoshan/compare_emis/GRIDCRO2D_2022234.nc')

In [4]:
# 将时间点转换为实际时间
times=pd.date_range('2022-09-02-00','2022-09-07-23',freq='h')
times

DatetimeIndex(['2022-09-02 00:00:00', '2022-09-02 01:00:00',
               '2022-09-02 02:00:00', '2022-09-02 03:00:00',
               '2022-09-02 04:00:00', '2022-09-02 05:00:00',
               '2022-09-02 06:00:00', '2022-09-02 07:00:00',
               '2022-09-02 08:00:00', '2022-09-02 09:00:00',
               ...
               '2022-09-07 14:00:00', '2022-09-07 15:00:00',
               '2022-09-07 16:00:00', '2022-09-07 17:00:00',
               '2022-09-07 18:00:00', '2022-09-07 19:00:00',
               '2022-09-07 20:00:00', '2022-09-07 21:00:00',
               '2022-09-07 22:00:00', '2022-09-07 23:00:00'],
              dtype='datetime64[ns]', length=144, freq='H')

In [5]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [6]:
# 结合地形高度HT与各气压层离地高度ZH获得各层实际海拔高度
ht=np.squeeze(grid.HT)
height=np.zeros([np.size(chem.ZH,0),
                 np.size(chem.ZH,1),
                 np.size(chem.ZH,2),
                 np.size(chem.ZH,3)])
for t in range(np.size(chem.ZH,0)):
    for l in range(np.size(chem.ZH,1)):
        height[t,l,:,:]=chem.ZH[t,l,:,:]+ht

In [11]:
days=1 # set spin-up days
dataset=xr.Dataset(
    data_vars=dict(
        # ! vars from CMAQ
        CO=(['time','level','y','x'],chem.CO[days*24-8:-8,:,:,:].data*30/22.4,{'long name':'Carbon Monoxide','units':'ug m-3'}),
        O3=(['time','level','y','x'],chem.O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Ozone','units':'ug m-3'}),
        NO=(['time','level','y','x'],chem.NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Nitric Oxide','units':'ug m-3'}),
        NO2=(['time','level','y','x'],chem.NO2[days*24-8:-8,:,:,:].data*37/22.4,{'long name':'Nitrogen Dioxide','units':'ug m-3'}),
        SO2=(['time','level','y','x'],chem.SO2[days*24-8:-8,:,:,:].data*64/22.4,{'long name':'Sulfur Dioxide','units':'ug m-3'}),
        VOC=(['time','level','y','x'],chem.VOC[days*24-8:-8,:,:,:].data,{'long name':'Volitile Organic Compounds','units':'ppbC'}),
        PM25=(['time','level','y','x'],chem.PM25_TOT[days*24-8:-8,:,:,:].data,{'long name':'','units':'ug m-3'}),
        PM10=(['time','level','y','x'],chem.PM10[days*24-8:-8,:,:,:].data,{'long name':'','units':'ug m-3'}),
        # ! wwind
        wwind=(['time','level','y','x'],chem.WWind[days*24-8:-8,:,:,:].data,{'long name':'Vertical Wind Speed','units':'m s-1'}),
        # ! altitude
        HT=(['time','level','y','x'],height[days*24-8:-8,:,:,:].data,{'long name':'Altitudes','units':'m'}),
        ),
    coords=dict(
        time=times,
        level=pres,
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Xiaoshan_202209',
        grid='CN3XS_135X138',
        create=pd.Timestamp('now').strftime('%Y-%m-%d')
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (time: 144, level: 38, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2022-09-02 ... 2022-09-07T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 135.8 101.5 72.89
    latitude   (y, x) float32 28.29 28.29 28.29 28.29 ... 31.89 31.89 31.89
    longitude  (y, x) float32 117.9 117.9 117.9 118.0 ... 122.3 122.3 122.4
Dimensions without coordinates: y, x
Data variables:
    CO         (time, level, y, x) float32 334.8 333.5 333.8 ... 36.14 36.15
    O3         (time, level, y, x) float32 108.0 108.3 ... 2.748e+03 2.748e+03
    NO         (time, level, y, x) float32 0.001765 0.0009917 ... 2.478e-09
    NO2        (time, level, y, x) float32 11.47 10.82 10.66 ... 0.07521 0.07543
    SO2        (time, level, y, x) float32 8.53 8.311 ... 0.002672 0.002672
    VOC        (time, level, y, x) float32 426.7 528.8 702.6 ... 0.01436 0.01438
    PM25       (time, level, y, x) float32 32.5 31.55 31.15 ... 0.0 0.0 0.0
    PM10       (time, level, y, x) float32 37.77 36.65 36.16 ... 0.0 0.0 0.0
    wwind      (time, level, y, x) float32 -0.02694 -0.01936 ... 0.004985
    HT         (time, level, y, x) float64 115.4 203.2 ... 1.969e+04 1.969e+04
Attributes:
    case:     Xiaoshan_202209
    grid:     CN3XS_135X138
    create:   2023-06-20

In [12]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset.data_vars}
dataset.to_netcdf('D:/Download/XS_heic.nc',encoding=encoding)